In [26]:
import numpy as np
import pandas as pd
import re
import copy
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
dir_path = 'drive/MyDrive/Data/linking-writing-processes-to-writing-quality/'

In [3]:
'''testlog0 = pd.read_csv(dir_path+'test_logs.csv')
trainlog0 = pd.read_csv(dir_path+'train_logs.csv')
trainscore0 = pd.read_csv(dir_path+'train_scores.csv')'''
testlog = pd.read_csv(dir_path+'test_logs.csv')
trainlog = pd.read_csv(dir_path+'train_logs.csv')
trainscore = pd.read_csv(dir_path+'train_scores.csv')


In [24]:
'''testlog = testlog0.copy()
trainlog = trainlog0.copy()
trainscore = trainscore0.copy()'''

'testlog = testlog0.copy()\ntrainlog = trainlog0.copy()\ntrainscore = trainscore0.copy()'

In [4]:
trainlog.head()

,id,event_id,down_time,up_time,action_time,activity,down_event,up_event,text_change,cursor_position,word_count
0,001519c8,1,4526,4557,31,Nonproduction,Leftclick,Leftclick,NoChange,0,0
1,001519c8,2,4558,4962,404,Nonproduction,Leftclick,Leftclick,NoChange,0,0
2,001519c8,3,106571,106571,0,Nonproduction,Shift,Shift,NoChange,0,0
3,001519c8,4,106686,106777,91,Input,q,q,q,1,1
4,001519c8,5,107196,107323,127,Input,q,q,q,2,1


In [5]:
testlog.head()

,id,event_id,down_time,up_time,action_time,activity,down_event,up_event,text_change,cursor_position,word_count
0,0000aaaa,1,338433,338518,85,Input,Space,Space,,0,0
1,0000aaaa,2,760073,760160,87,Input,Space,Space,,1,0
2,2222bbbb,1,711956,712023,67,Input,q,q,q,0,1
3,2222bbbb,2,290502,290548,46,Input,q,q,q,1,1
4,4444cccc,1,635547,635641,94,Input,Space,Space,,0,0


有534行出现了downevent!=upevent的情况。为什么？什么情况？按下键盘之后松开的还能不是一个键了？

In [27]:
# 找到 down_event 列值与 up_event 列值不同的行
result = trainlog[trainlog['down_event'] != trainlog['up_event']]

# 打印结果
result.head()

,id,event_id,down_time,up_time,action_time,activity,down_event,up_event,text_change,cursor_position,word_count
5115,0042269b,105,104507,104617,110,Replace,q,m,qqqqqqq qqq qqqq qqqq qqqq qq qqqqqqqq qqqqqqq...,1,4
5232,0042269b,222,158578,158638,60,Replace,q,i,qqqqqqq => q,15,14
6276,0042269b,1266,473329,473439,110,Replace,q,o,qqq qq qqq qqqqqqq qqqqqq qqqq qq => q,194,128
6825,0042269b,1815,641821,641920,99,Replace,q,t,qqq qqqqq qq qqqqqqqq qqqq qqq qqqqqqq qq qqqq...,529,190
8223,0042269b,3213,1410198,1410293,95,Replace,q,a,qqqqqqq qqqqqqqqq qqqq qqq qq qqqq qqq qqqqqqq...,1539,282


In [28]:
# 将结果输出到 CSV 文件
result.to_csv(dir_path+'downup.csv', index=False)

提取之后发现这些downevent!=upevent的行中，upevent都是某些字母或乱码，则合理推测这是数据处理失误，应该是所有行动中的downevent都等于upevent才对

增加delta_cursor列，记录一次光标移动距离

In [8]:
# 计算每行的 delta_cursor 值
trainlog['delta_cursor'] = (trainlog['cursor_position'] - trainlog['cursor_position'].shift(1))
trainlog.loc[0, 'delta_cursor'] = 0

增加movedis列，记录一次文本转移距离，增加movelength列，记录一次文本转移长度

In [9]:
# 自定义函数
def set_movedis_movelength(row):
    if re.search(r'Move From \[(\d+), (\d+)\] To \[(\d+), (\d+)\]', row['activity']):
        match = re.search(r'Move From \[(\d+), (\d+)\] To \[(\d+), (\d+)\]', row['activity'])
        x1, y1, x2, y2 = map(int, match.groups())
        dx = x2 - x1
        dy = y2 - y1
        dis = abs(x2 - x1)
        length = (y1 - x1)
    else:
        dis = 0
        length = 0
    return dis, length

# 增加列 dx 和 dy
trainlog['movedis'], trainlog['movelength'] = zip(*trainlog.apply(set_movedis_movelength, axis=1))

增加pastelength列，记录黏贴长度

In [10]:
# 定义一个函数来设置 pastelength 的值
def set_pastelength(row):
    if row['activity'] == 'Paste':
        return len(row['text_change'])
    else:
        return 0

# 使用 apply() 方法应用函数，并将结果存储在 pastelength 列中
trainlog['pastelength'] = trainlog.apply(set_pastelength, axis=1)

增加replacelength列，记录每次文本替换所替换的字母数量差距

In [11]:
# 定义一个函数来设置 replacelength 的值
def set_replacelength(row):
    if row['activity'] == 'Replace':
        # 获取替换前和替换后的字符串
        replace_text = row['text_change']
        before_text, after_text = replace_text.split(' => ')

        # 计算替换前和替换后的长度（包括空格）
        before_length = len(before_text)
        after_length = len(after_text)

        # 返回替换前和替换后长度的差值
        return before_length - after_length
    else:
        return 0

# 使用 apply() 方法应用函数，并将结果存储在 replacelength 列中
trainlog['replacelength'] = trainlog.apply(set_replacelength, axis=1)

增加RClength列，记录Remove/Cut行动删去的字符数量

In [12]:
# 定义一个函数来设置 RClength 的值
def set_RClength(row):
    if row['activity'] == 'Remove/Cut':
        text_change = row['text_change']
        return len(text_change)
    else:
        return 0

# 使用 apply() 方法应用函数，并将结果存储在 RClength 列中
trainlog['RClength'] = trainlog.apply(set_RClength, axis=1)

处理down_event列，整理错误的字母输入

In [13]:
trainlog = trainlog.drop("up_event", axis=1)#合理假设所有按下按键的按键名就是释放按键时的按键名

In [14]:
Keyboard_keywords = ['Leftclick', 'Shift', 'Backspace', 'Enter', 'ArrowLeft', 'ArrowRight', 'Tab', 'ArrowUp', 'ArrowDown', 'Rightclick',
           'CapsLock', 'Control', 'Delete', 'Home', 'End', 'Insert', 'NumLock', 'Alt',
           'ContextMenu','PageDown', 'Middleclick', 'ScrollLock', 'Space',
           'Cancel', 'Escape', 'Clear', 'OS']
for i in range(1,16):
  Keyboard_keywords.append('F'+str(i))#F1--F15
Media_keywords = ['Meta','AudioVolumeMute','MediaPlayPause','AudioVolumeUp','AudioVolumeDown','MediaTrackPrevious', 'MediaTrackNext', 'Pause']
Other_keywords = ['Dead', 'Process', 'AltGraph',]
Unknown_keywords = ['Unidentified','Unknownclick']

In [15]:
# 定义一个函数来转换 down_event 列的值
trainlog['keyword'] = 0
def convert_down_event_char(value):
    if len(value) == 1 and (value.isalpha() or value.isdigit()):
        return 'q'
    elif value in Keyboard_keywords:
        return "keyboard_keyword"
    elif value in Media_keywords:
        return "media_keyword"
    elif value in Other_keywords:
        return "other_keyword"
    elif value in Unknown_keywords:
        return "unknown_keyword"
    else:
        return "unknown_keyword"

# 使用 apply() 方法应用函数，并将结果存储回 down_event 列中
trainlog['keyword'] = trainlog['down_event'].apply(convert_down_event_char)

处理6种可能的行动

In [16]:
actype = {0:'Nonproduction',
      1:'Input',
      2:'Remove/Cut',
      3:'Paste',
      4:'Replace',
      5:'Move'
      }

In [17]:
# 定义一个函数来设置 astype 的值
def set_activity(row):
    if row['activity'] == 'Nonproduction':
        return 0
    elif row['activity'] == 'Input':
        return 1
    elif row['activity'] == 'Remove/Cut':
        return 2
    elif row['activity'] == 'Paste':
        return 3
    elif row['activity'] == 'Replace':
        return 4
    else:
      return 5

# 使用 apply() 方法应用函数，并将结果存储在 astype 列中
trainlog['actype'] = trainlog.apply(set_activity,axis=1)

增加两列用于记录连打情况以及思考时间，上一次按键释放到下一个按键按下间的时间间隔视为思考时间，上一个按键释放前下一个按键已经按下则视为一次连打

In [18]:
#trainlog = trainlog.drop("thinktime", axis=1)
#trainlog = trainlog.drop("contype", axis=1)

In [19]:
# 新增两列用于存储思考时间和连打标记
trainlog['thinktime'] = 0
trainlog['contype'] = 0

# 计算思考时间
trainlog['thinktime'] = trainlog['down_time'] - trainlog['up_time'].shift()
trainlog['thinktime'] = trainlog['thinktime'].clip(lower=0)

# 标记连打情况
trainlog['contype'] = (trainlog['down_time'] <= trainlog['up_time'].shift()).astype(int)

In [20]:
trainlog = trainlog.drop("activity", axis=1)
trainlog = trainlog.drop("down_event", axis=1)
trainlog = trainlog.drop("text_change", axis=1)
trainlog = trainlog.drop("cursor_position", axis=1)

In [21]:
trainlog.to_csv(dir_path+'trainlog1.csv', index=False)

In [22]:
result = trainlog[trainlog['thinktime'] > 1]
result.head()

,id,event_id,down_time,up_time,action_time,word_count,delta_cursor,movedis,movelength,pastelength,replacelength,RClength,keyword,actype,thinktime,contype
2,001519c8,3,106571,106571,0,0,0.0,0,0,0,0,0,keyboard_keyword,0,101609.0,0
3,001519c8,4,106686,106777,91,1,1.0,0,0,0,0,0,q,1,115.0,0
4,001519c8,5,107196,107323,127,1,1.0,0,0,0,0,0,q,1,419.0,0
6,001519c8,7,107469,107596,127,1,1.0,0,0,0,0,0,q,1,69.0,0
7,001519c8,8,107659,107766,107,1,1.0,0,0,0,0,0,q,1,63.0,0


- id - 文章的唯一ID
- event_id - 事件的索引，按时间顺序排序
- down_time - 按下事件的时间（毫秒）
- up_time - 松开事件的时间（毫秒）
- action_time - 事件的持续时间（`down_time`和`up_time`之间的差值）
- delta_cursor - 每次行动后光标移动的距离
- movedis - Move操作中移动的距离
- movelength - Move操作中移动的字符串长度
- pastelength - 一次黏贴的字符串长度
- replacelength - 一次replace操作替换的字符串长度差值
- RClength - 一次Remove/Cut操作删去的字符串长度
- thinktime - 思考时间
- contype - 标记是否为连打
- actype:操作类型:
      0:Nonproduction,
      1:Input,
      2:Remove/Cut,
      3:Paste,
      4:Replace,
      5:Move


In [ ]:
#用于读取之前处理好的数据
#trainlog = pd.read_csv(dir_path+'trainlog1.csv')

In [28]:
# 创建新的dataframe变量traindata
traindata = pd.DataFrame()

# 以id作为标识遍历trainlog
for id_value, group in tqdm(trainlog.groupby('id')):
    word_count = group['word_count'].iloc[-1]
    # 统计连打次数
    contype_count = group['contype'].eq(1).sum()

    # 统计思考总时长
    total_think = group['thinktime'].sum()

    # 统计总用时
    total_time = group['up_time'].iloc[-1] - group['down_time'].iloc[0]

    # 统计所有操作数量
    action_count = group.shape[0]

    # 统计所有操作总用时
    total_actiontime = group['action_time'].sum()

    # 统计keyword列中各值出现的次数
    keyword_counts = group['keyword'].value_counts()
    keyboard_count = keyword_counts.get('keyboard_keyword', 0)
    media_count = keyword_counts.get('media_keyword', 0)
    other_count = keyword_counts.get('other_keyword', 0)
    unknown_count = keyword_counts.get('unknown_keyword', 0)

    # 统计delta_cursor列中各区间内的数目
    delta_cursor = group['delta_cursor']
    dc0 = delta_cursor.between(0, 1).sum()
    dc1 = delta_cursor.between(2, 10).sum()
    dc2 = delta_cursor.between(11, 30).sum()
    dc3 = delta_cursor.between(31, 80).sum()
    dc4 = (delta_cursor > 80).sum()
    dc5 = delta_cursor.between(-1, -1).sum()
    dc6 = delta_cursor.between(-10, -2).sum()
    dc7 = delta_cursor.between(-30, -11).sum()
    dc8 = delta_cursor.between(-80, -31).sum()
    dc9 = (delta_cursor < -80).sum()

    # 统计movedis列中各区间内的数目
    movedis = group['movedis']
    md0 = movedis.between(0, 1).sum()
    md1 = movedis.between(2, 10).sum()
    md2 = movedis.between(11, 30).sum()
    md3 = movedis.between(31, 80).sum()
    md4 = (movedis > 80).sum()

    # 统计movelength列中各区间内的数目
    movelength = group['movelength']
    ml0 = movelength.between(0, 5).sum()
    ml1 = movelength.between(6, 15).sum()
    ml2 = movelength.between(16, 30).sum()
    ml3 = movelength.between(31, 80).sum()
    ml4 = (movelength > 80).sum()

    # 统计pastelength列中各区间内的数目
    pastelength = group['pastelength']
    pl0 = pastelength.between(0, 1).sum()
    pl1 = pastelength.between(2, 10).sum()
    pl2 = pastelength.between(11, 30).sum()
    pl3 = pastelength.between(31, 80).sum()
    pl4 = (pastelength > 80).sum()

    # 统计replacelength列中各区间内的数目
    replacelength = group['replacelength']
    rl0 = replacelength.between(0, 1).sum()
    rl1 = replacelength.between(2, 10).sum()
    rl2 = replacelength.between(11, 30).sum()
    rl3 = replacelength.between(31, 80).sum()
    rl4 = (replacelength > 80).sum()

    # 统计RClength列中各区间内的数目
    RClength = group['RClength']
    rc0 = RClength.between(0, 1).sum()
    rc1 = RClength.between(2, 10).sum()
    rc2 = RClength.between(11, 30).sum()
    rc3 = RClength.between(31, 80).sum()
    rc4 = (RClength > 80).sum()

    # 统计actype列中各值出现的次数
    actype_counts = group['actype'].value_counts()
    ac0 = actype_counts.get(0, 0)
    ac1 = actype_counts.get(1, 0)
    ac2 = actype_counts.get(2, 0)
    ac3 = actype_counts.get(3, 0)
    ac4 = actype_counts.get(4, 0)
    ac5 = actype_counts.get(5, 0)

    # 统计连续输入次数
    consecutive_input = np.where(
        (group['actype'] == 1) & (group['actype'].shift(1) != 1), 1, 0
    )
    consecutive_input_count = consecutive_input.sum()

    # 存储信息到traindata中
    traindata = traindata.append(
        {
            'id': id_value,
            'word_count': word_count,
            'contype_count': contype_count,
            'total_think': total_think,
            'total_time': total_time,
            'action_count': action_count,
            'total_actiontime': total_actiontime,
            'keyboard_count': keyboard_count,
            'media_count': media_count,
            'other_count': other_count,
            'unknown_count': unknown_count,
            'dc0': dc0,
            'dc1': dc1,
            'dc2': dc2,
            'dc3': dc3,
            'dc4': dc4,
            'dc5': dc5,
            'dc6': dc6,
            'dc7': dc7,
            'dc8': dc8,
            'dc9': dc9,
            'md0': md0,
            'md1': md1,
            'md2': md2,
            'md3': md3,
            'md4': md4,
            'ml0': ml0,
            'ml1': ml1,
            'ml2': ml2,
            'ml3': ml3,
            'ml4': ml4,
            'pl0': pl0,
            'pl1': pl1,
            'pl2': pl2,
            'pl3': pl3,
            'pl4': pl4,
            'rl0': rl0,
            'rl1': rl1,
            'rl2': rl2,
            'rl3': rl3,
            'rl4': rl4,
            'rc0': rc0,
            'rc1': rc1,
            'rc2': rc2,
            'rc3': rc3,
            'rc4': rc4,
            'ac0': ac0,
            'ac1': ac1,
            'ac2': ac2,
            'ac3': ac3,
            'ac4': ac4,
            'ac5': ac5,
            'consecutive_input_count': consecutive_input_count,
        },
        ignore_index=True
    )

100%|██████████| 2471/2471 [00:58<00:00, 42.52it/s]


In [31]:
traindata.to_csv(dir_path+'traindata.csv', index=False)

# 下面试着将所有数据处理集中至一个函数中，首先是所有的自定义函数

In [49]:
def set_movedis_movelength(row):
    if re.search(r'Move From \[(\d+), (\d+)\] To \[(\d+), (\d+)\]', row['activity']):
        match = re.search(r'Move From \[(\d+), (\d+)\] To \[(\d+), (\d+)\]', row['activity'])
        x1, y1, x2, y2 = map(int, match.groups())
        dx = x2 - x1
        dy = y2 - y1
        dis = abs(x2 - x1)
        length = (y1 - x1)
    else:
        dis = 0
        length = 0
    return dis, length

def set_pastelength(row):
    if row['activity'] == 'Paste':
        return len(row['text_change'])
    else:
        return 0

def set_replacelength(row):
    if row['activity'] == 'Replace':
        replace_text = row['text_change']
        before_text, after_text = replace_text.split(' => ')

        before_length = len(before_text)
        after_length = len(after_text)

        return before_length - after_length
    else:
        return 0

def set_RClength(row):
    if row['activity'] == 'Remove/Cut':
        text_change = row['text_change']
        return len(text_change)
    else:
        return 0

Keyboard_keywords = ['Leftclick', 'Shift', 'Backspace', 'Enter', 'ArrowLeft', 'ArrowRight', 'Tab', 'ArrowUp', 'ArrowDown', 'Rightclick',
           'CapsLock', 'Control', 'Delete', 'Home', 'End', 'Insert', 'NumLock', 'Alt',
           'ContextMenu','PageDown', 'Middleclick', 'ScrollLock', 'Space',
           'Cancel', 'Escape', 'Clear', 'OS']
for i in range(1,16):
  Keyboard_keywords.append('F'+str(i))#F1--F15
Media_keywords = ['Meta','AudioVolumeMute','MediaPlayPause','AudioVolumeUp','AudioVolumeDown','MediaTrackPrevious', 'MediaTrackNext', 'Pause']
Other_keywords = ['Dead', 'Process', 'AltGraph',]
Unknown_keywords = ['Unidentified','Unknownclick']

def convert_down_event_char(value):
    if len(value) == 1 and (value.isalpha() or value.isdigit()):
        return 'q'
    elif value in Keyboard_keywords:
        return "keyboard_keyword"
    elif value in Media_keywords:
        return "media_keyword"
    elif value in Other_keywords:
        return "other_keyword"
    elif value in Unknown_keywords:
        return "unknown_keyword"
    else:
        return "unknown_keyword"

actype = {0:'Nonproduction',
      1:'Input',
      2:'Remove/Cut',
      3:'Paste',
      4:'Replace',
      5:'Move'
      }

def set_activity(row):
    if row['activity'] == 'Nonproduction':
        return 0
    elif row['activity'] == 'Input':
        return 1
    elif row['activity'] == 'Remove/Cut':
        return 2
    elif row['activity'] == 'Paste':
        return 3
    elif row['activity'] == 'Replace':
        return 4
    else:
      return 5

# 预处理

In [99]:
def preprocess(df):

  df.sort_values(by=['id', 'event_id'])

  df['delta_cursor'] = (df['cursor_position'] - df['cursor_position'].shift(1))
  df.loc[0, 'delta_cursor'] = 0

  df['movedis'], df['movelength'] = zip(*df.apply(set_movedis_movelength, axis=1))

  df['pastelength'] = df.apply(set_pastelength, axis=1)

  df['replacelength'] = df.apply(set_replacelength, axis=1)

  df['RClength'] = df.apply(set_RClength, axis=1)

  df['keyword'] = df['down_event'].apply(convert_down_event_char)

  df['actype'] = df.apply(set_activity,axis=1)

  df['thinktime'] = df['down_time'] - df['up_time'].shift()
  df['thinktime'] = df['thinktime'].clip(lower=0)

  df['contype'] = (df['down_time'] <= df['up_time'].shift()).astype(int)

  df.drop(["up_event", "activity", "down_event", "text_change", "cursor_position"], axis=1, inplace=True)

# 提取特征

In [112]:
def getfeature(df,data):
  consecutive_input_lengths = []
  for id_value, group in tqdm(df.groupby('id')):
    #print(id_value)
    word_count = group['word_count'].iloc[-1]

    contype_count = group['contype'].eq(1).sum()

    total_think = group['thinktime'].sum()

    total_time = group['up_time'].iloc[-1] - group['down_time'].iloc[0]

    action_count = group.shape[0]

    total_actiontime = group['action_time'].sum()

    keyword_counts = group['keyword'].value_counts()
    keyboard_count = keyword_counts.get('keyboard_keyword', 0)
    media_count = keyword_counts.get('media_keyword', 0)
    other_count = keyword_counts.get('other_keyword', 0)
    unknown_count = keyword_counts.get('unknown_keyword', 0)

    delta_cursor = group['delta_cursor']
    dc0 = delta_cursor.between(0, 1).sum()
    dc1 = delta_cursor.between(2, 10).sum()
    dc2 = delta_cursor.between(11, 30).sum()
    dc3 = delta_cursor.between(31, 80).sum()
    dc4 = (delta_cursor > 80).sum()
    dc5 = delta_cursor.between(-1, -1).sum()
    dc6 = delta_cursor.between(-10, -2).sum()
    dc7 = delta_cursor.between(-30, -11).sum()
    dc8 = delta_cursor.between(-80, -31).sum()
    dc9 = (delta_cursor < -80).sum()

    movedis = group['movedis']
    md0 = movedis.between(0, 1).sum()
    md1 = movedis.between(2, 10).sum()
    md2 = movedis.between(11, 30).sum()
    md3 = movedis.between(31, 80).sum()
    md4 = (movedis > 80).sum()

    movelength = group['movelength']
    ml0 = movelength.between(0, 5).sum()
    ml1 = movelength.between(6, 15).sum()
    ml2 = movelength.between(16, 30).sum()
    ml3 = movelength.between(31, 80).sum()
    ml4 = (movelength > 80).sum()

    pastelength = group['pastelength']
    pl0 = pastelength.between(0, 1).sum()
    pl1 = pastelength.between(2, 10).sum()
    pl2 = pastelength.between(11, 30).sum()
    pl3 = pastelength.between(31, 80).sum()
    pl4 = (pastelength > 80).sum()

    replacelength = group['replacelength']
    rl0 = replacelength.between(0, 1).sum()
    rl1 = replacelength.between(2, 10).sum()
    rl2 = replacelength.between(11, 30).sum()
    rl3 = replacelength.between(31, 80).sum()
    rl4 = (replacelength > 80).sum()

    RClength = group['RClength']
    rc0 = RClength.between(0, 1).sum()
    rc1 = RClength.between(2, 10).sum()
    rc2 = RClength.between(11, 30).sum()
    rc3 = RClength.between(31, 80).sum()
    rc4 = (RClength > 80).sum()

    actype_counts = group['actype'].value_counts()
    ac0 = actype_counts.get(0, 0)
    ac1 = actype_counts.get(1, 0)
    ac2 = actype_counts.get(2, 0)
    ac3 = actype_counts.get(3, 0)
    ac4 = actype_counts.get(4, 0)
    ac5 = actype_counts.get(5, 0)

    '''consecutive_input = np.where(
        (group['actype'] == 1) & (group['actype'].shift(1) != 1), 1, 0
    )
    consecutive_input_count = consecutive_input.sum()'''

    consecutive_input_count = 0
    consecutive_input_lengths = [1]
    prev_action_type = None
    for index, row in group.iterrows():
      current_action_type = row['actype']
      if prev_action_type == 1 and current_action_type != 1:
          consecutive_input_count += 1
          consecutive_input_lengths.append(0)
      elif prev_action_type == 1 and current_action_type == 1:
          consecutive_input_lengths[-1] += 1
      prev_action_type = current_action_type

    consecutive_input_max = max(consecutive_input_lengths)
    consecutive_input_mean = np.mean(consecutive_input_lengths)
    consecutive_input_median = np.median(consecutive_input_lengths)
    consecutive_input_variance = np.var(consecutive_input_lengths)

    data = data.append(
        {
            'id': id_value,
            'word_count': word_count,
            'contype_count': contype_count,
            'total_think': total_think,
            'total_time': total_time,
            'action_count': action_count,
            'total_actiontime': total_actiontime,
            'keyboard_count': keyboard_count,
            'media_count': media_count,
            'other_count': other_count,
            'unknown_count': unknown_count,
            'dc0': dc0,
            'dc1': dc1,
            'dc2': dc2,
            'dc3': dc3,
            'dc4': dc4,
            'dc5': dc5,
            'dc6': dc6,
            'dc7': dc7,
            'dc8': dc8,
            'dc9': dc9,
            'md0': md0,
            'md1': md1,
            'md2': md2,
            'md3': md3,
            'md4': md4,
            'ml0': ml0,
            'ml1': ml1,
            'ml2': ml2,
            'ml3': ml3,
            'ml4': ml4,
            'pl0': pl0,
            'pl1': pl1,
            'pl2': pl2,
            'pl3': pl3,
            'pl4': pl4,
            'rl0': rl0,
            'rl1': rl1,
            'rl2': rl2,
            'rl3': rl3,
            'rl4': rl4,
            'rc0': rc0,
            'rc1': rc1,
            'rc2': rc2,
            'rc3': rc3,
            'rc4': rc4,
            'ac0': ac0,
            'ac1': ac1,
            'ac2': ac2,
            'ac3': ac3,
            'ac4': ac4,
            'ac5': ac5,
            'consecutive_input_count': consecutive_input_count,
            'consecutive_input_max': consecutive_input_max,
            'consecutive_input_mean': consecutive_input_mean,
            'consecutive_input_mid': consecutive_input_median,
            'consecutive_input_var': consecutive_input_variance,
        },
        ignore_index=True
    )
  return data


In [113]:
testlog = pd.read_csv(dir_path+'test_logs.csv')
testlog.head(10)

,id,event_id,down_time,up_time,action_time,activity,down_event,up_event,text_change,cursor_position,word_count
0,0000aaaa,1,338433,338518,85,Input,Space,Space,,0,0
1,0000aaaa,2,760073,760160,87,Input,Space,Space,,1,0
2,2222bbbb,1,711956,712023,67,Input,q,q,q,0,1
3,2222bbbb,2,290502,290548,46,Input,q,q,q,1,1
4,4444cccc,1,635547,635641,94,Input,Space,Space,,0,0
5,4444cccc,2,184996,185052,56,Input,q,q,q,1,1


In [114]:
testdata = pd.DataFrame()
preprocess(testlog)
testdata = getfeature(testlog,testdata)

100%|██████████| 3/3 [00:00<00:00, 53.64it/s]


In [116]:
testdata.head()

,id,word_count,contype_count,total_think,total_time,action_count,total_actiontime,keyboard_count,media_count,other_count,...,ac1,ac2,ac3,ac4,ac5,consecutive_input_count,consecutive_input_max,consecutive_input_mean,consecutive_input_mid,consecutive_input_var
0,0000aaaa,0,0,421555.0,421727,2,172,2,0,0,...,2,0,0,0,0,0,2,2.0,2.0,0.0
1,2222bbbb,1,2,0.0,-421408,2,113,0,0,0,...,2,0,0,0,0,0,2,2.0,2.0,0.0
2,4444cccc,1,1,344999.0,-450495,2,150,1,0,0,...,2,0,0,0,0,0,2,2.0,2.0,0.0


In [117]:
trainlog = pd.read_csv(dir_path+'train_logs.csv')
preprocess(trainlog)
traindata1 = pd.DataFrame()
traindata1 = getfeature(trainlog, traindata1)

100%|██████████| 2471/2471 [08:43<00:00,  4.72it/s]


In [118]:
traindata.head(10)

,id,word_count,contype_count,total_think,total_time,action_count,total_actiontime,keyboard_count,media_count,other_count,...,rc2,rc3,rc4,ac0,ac1,ac2,ac3,ac4,ac5,consecutive_input_count
0,001519c8,255,901,1531192.0,1797443,2557,297243,544,0,0,...,0,0,0,120,2010,417,0,7,3,124
1,0022f953,320,616,1502350.0,1758346,2454,275391,520,0,0,...,0,0,0,254,1938,260,1,1,0,122
2,0042269b,404,2188,1418539.0,1767228,4136,421201,631,0,0,...,2,0,1,175,3515,439,0,7,0,149
3,0059420b,206,206,1183046.0,1363074,1556,189596,248,0,0,...,0,0,0,99,1304,151,1,1,0,80
4,0075873a,252,1042,1297666.0,1584002,2531,313702,599,0,0,...,0,0,0,72,1942,517,0,0,0,83
5,0081af50,275,224,1561037.0,1736280,2211,179985,417,0,0,...,0,0,0,76,1794,338,0,3,0,139
6,0093f095,241,480,1581005.0,1761625,1765,193652,186,0,0,...,0,0,0,34,1583,148,0,0,0,81
7,009e23ab,307,545,1492563.0,1692754,2353,213548,386,0,0,...,0,0,0,155,1975,222,0,1,0,93
8,00e048f1,223,24,1673375.0,1792225,1585,118893,164,0,0,...,0,0,0,41,1426,118,0,0,0,63
9,00e1f05a,739,2149,1187053.0,1842966,7826,730504,1674,0,0,...,0,4,1,228,6145,1446,0,7,0,347


In [119]:
traindata1.head(10)

,id,word_count,contype_count,total_think,total_time,action_count,total_actiontime,keyboard_count,media_count,other_count,...,ac1,ac2,ac3,ac4,ac5,consecutive_input_count,consecutive_input_max,consecutive_input_mean,consecutive_input_mid,consecutive_input_var
0,001519c8,255,901,1531192.0,1797443,2557,297243,901,0,0,...,2010,417,0,7,3,124,81,15.096000,9.0,308.966784
1,0022f953,320,616,1502350.0,1758346,2454,275391,911,0,0,...,1938,260,1,1,0,122,90,14.772358,7.0,347.053870
2,0042269b,404,2188,1418539.0,1767228,4136,421201,1183,0,0,...,3515,439,0,7,0,149,118,22.446667,16.5,528.220489
3,0059420b,206,206,1183046.0,1363074,1556,189596,491,0,0,...,1304,151,1,1,0,79,71,15.312500,10.0,251.839844
4,0075873a,252,1042,1297666.0,1584002,2531,313702,923,0,0,...,1942,517,0,0,0,82,127,22.409639,15.0,684.675570
5,0081af50,275,224,1561037.0,1736280,2211,179985,744,0,0,...,1794,338,0,3,0,138,86,11.913669,7.0,178.640029
6,0093f095,241,480,1581005.0,1761625,1765,193652,443,0,0,...,1583,148,0,0,0,81,93,18.329268,8.0,567.684265
7,009e23ab,307,545,1492563.0,1692754,2353,213548,752,0,0,...,1975,222,0,1,0,93,139,20.031915,9.0,649.903237
8,00e048f1,223,24,1673375.0,1792225,1585,118893,409,0,0,...,1426,118,0,0,0,63,109,21.312500,16.0,398.058594
9,00e1f05a,739,2149,1187053.0,1842966,7826,730504,2761,0,0,...,6145,1446,0,7,0,347,139,16.663793,11.0,401.740413


In [121]:
traindata1.to_csv(dir_path+'traindata_v1.csv',index=False)